In [1]:
import pandas as pd
import re
import os
import csv
import sys
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

/opt/conda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Import Data

In [2]:
#Load Data and split 
X_train = pd.read_csv('train.csv')
X_val = pd.read_csv('validation.csv')
y_train = X_train['click']
y_val = X_val['click']
print("finshed importing training data")


plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', None) # Display any number of columns

finshed importing training data


In [3]:
from sklearn.feature_extraction import DictVectorizer

v = DictVectorizer(sparse=True)

def process_train(df):
    drop = ['logtype', 'bidid', 'url', 'payprice', 'bidprice', 'keypage', 'urlid', 'click']
    df.drop(drop,inplace=True, axis=1)
    df = df[['weekday', 'hour', 'region', 'adexchange','advertiser','slotvisibility']]
    df = df.to_dict(orient='records')
    X = v.fit_transform
    return X

def process_test(df):
    drop = ['logtype', 'bidid', 'url', 'payprice', 'bidprice', 'keypage', 'urlid', 'click']
    df.drop(drop,inplace=True,axis=1)
    df = df[['weekday', 'hour', 'region', 'adexchange','advertiser','slotvisibility']]
    df = df.to_dict(orient='records')
    X = v.transform(df)
    return X


In [5]:
#Vectorize and train
from sklearn.naive_bayes import GaussianNB
X = process_train(X_train)
expected = y_val
clf = GaussianNB()
clf.fit(X, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=0, splitter='best')

In [7]:
x_val = process_train(X_val)
predicted = predict_proba(x_val)
prediction = pd.DataFrame(predicted)
prediction.to_csv('NaiveBayes_1.csv')

In [31]:
y_pred = pd.read_csv('NaiveBayes_1.csv') 
#y_pred = pd.read_csv('click_pred_l1reg.txt')
y_pred = y_pred[' probability_click']
y_val = y_val.astype('float')
y_pred =y_pred.astype('float')
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss

def scores(y_true, y_pred):
    print("AUC: " + str(roc_auc_score(y_true, y_pred)))
    print("Cross Entropy" + str(log_loss(y_true, y_pred)))
    return print('Completed.')

scores(y_val, y_pred)
    

AUC: 0.615751626207
Cross Entropy0.369145572582
Completed.


In [ ]:
print("Started encoding x training data!")

print("Done encoding x training data!")
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
print("training model...")
model.fit(X, y_train)
print(model)

# Make Predictions

X_v = process_test(X_val)
expected = y_val
print("making predictions...")
predicted = model.predict(X_v)

metrics.confusion_matrix(expected, predicted)
print("total predictions:")
print(sum(predicted))

In [ ]:
####Alternative Training#####
"""
def process(df):
    
    drop = ['logtype', 'bidid', 'url', 'payprice', 'bidprice', 'keypage', 'urlid', 'click']
    df = df.drop(drop, 1)

    def slotprice_function(fp):
        if fp in range(0, 50):
            return 1
        elif fp in range (51, 100)
            return 2
        elif fp in range(101, 150):
            return 3
        elif fp in range(151, 200):
            return 4
        else:
            return 5

    df['slotprice'] = df['slotprice'].apply(slotprice_function)
    
    def gen_feature_set(df):
        #handle categorical variables
        label_encoder = preprocessing.LabelEncoder()

        df['useragent'] = label_encoder.fit_transform(df['useragent'])
        df['IP'] = label_encoder.fit_transform(df['IP'])
        df['domain'] = label_encoder.fit_transform(df['domain'])
        df['slotvisibility'] = label_encoder.fit_transform(df['slotvisibility'])
        df['adexchange'] = label_encoder.fit_transform(df['adexchange'])
        weekday = df['weekday']
        hour = df['hour']
        useragent = df['useragent']
        slotvisibility = df['slotvisibility']
        advertiser = df['advertiser']
        slotprice = df['slotprice']
        #IP = pd.get_dummies(train_d['advertiser'])
        region = df['region']
        city = df['city']
        #usertag_list = [len(x) for x in train_d['usertag'].tolist()]
        #max_user_taglist = max(usertag_list)
        #usertag = pd.DataFrame(train_d['usertag'].str.split(',', max_user_taglist ).tolist())
        domain = df['domain']
        adexchange = df['adexchange']
        IP = df['IP']
        
        features_dum = [weekday, hour, useragent, advertiser, region, city, domain,
                        adexchange, slotprice, IP, slotvisibility]
        data = pd.concat(features_dum, axis=1)
        print('Number of features ' + str(len(data.columns)))
        return data
    
    data = gen_feature_set(df)
    #data = data.to_dict(orient='index')
    return data

matrix = process(train_df)
X = matrix.as_matrix().astype(np.float)
lr = LogisticRegression(class_weight="auto")
lr.fit(X, y_train)

#x_test = vectorize(test_df)
x_test = process(test_df)
x_test = x_test.as_matrix().astype(np.float)
y_predict = lr.predict(x_test)
print('number of clicks predicted ' + str(sum(y_predict)))
print('number of clicks gold '+ str(sum(y_test)))
print('total: '+ str(len(y_test)))
"""